### read csv file
Your task is to read the input DATAFILE line by line, and for the first 10 lines (not including the header) split each line on "," and then for each line, create a dictionary. where the key is the header title of the field, and the value is the value of that field in the row. The function parse_file should return a list of dictionaries, each data line in the file being a single list entry. Field names and values should not contain extra whitespace, like spaces or newline characters. You can use the Python string method strip() to remove the extra whitespace. You have to parse only the first 10 data lines in this exercise, so the returned list should have 10 entries!

In [2]:
import os

def parse_file(datafile):
    data = []
    
    with open(datafile, "r") as f:
        header = f.readline().split(',')
        count = 0
        for line in f:
            if count == 10:
                break
            fields = line.split(',')
            entry = {}
            for i,value in enumerate(fields):
                entry[header[i].strip()] = value.strip()
            data.append(entry)
            count = count+1
    return data


In [5]:
import csv
with open("/Users/weidian1/Desktop/data.csv","r") as f:
    step = []
    accuracy = []
    for index,line in enumerate(f):
        value = line.split(":")[1]
        if index%2 == 0:
            step.append(float(value))
        else:
            accuracy.append(float(value[1:5])/100.)
with open('output.csv', 'wb') as csvfile:
    w = csv.writer(csvfile, delimiter=',')
    for i in range(len(step)):
        w.writerow([step[i] ,accuracy[i]])    

In [13]:
import csv
import pprint  #pprint.pprint  优美的输出
#import os
def parse_csv(datafile):
    data = []
    count = 0
    with open(datafile ,'rb') as sd:
        r = csv.DictReader(sd)
        for line in r :
            if count == 10:
                break
            data.append(line)
            count = count+1
    return data

In [14]:
DATADIR = "/Users/weidian1/Documents/Study/nanodegreee/P3"
DATAFILE = "beatles-diskography.csv"

def test(DATADIR,DATAFILE,parse_f):
    datafile = os.path.join(DATADIR, DATAFILE)
    d = parse_f(datafile)
    firstline = {'Title': 'Please Please Me', 'UK Chart Position': '1', 'Label': 'Parlophone(UK)', 'Released': '22 March 1963', 'US Chart Position': '-', 'RIAA Certification': 'Platinum', 'BPI Certification': 'Gold'}
    tenthline = {'Title': '', 'UK Chart Position': '1', 'Label': 'Parlophone(UK)', 'Released': '10 July 1964', 'US Chart Position': '-', 'RIAA Certification': '', 'BPI Certification': 'Gold'}
    #print d[0]
    assert d[0] == firstline
    assert d[9] == tenthline

    
test(DATADIR,DATAFILE,parse_f=parse_csv)

### write csv

In [13]:
import csv
with open('eggs.csv', 'wb') as csvfile:
    spamwriter = csv.writer(csvfile, delimiter=',')
                            #,quotechar='|', quoting=csv.QUOTE_MINIMAL)
    spamwriter.writerow(['Spam'] * 5 + ['Baked Beans'])
    spamwriter.writerow(['Spam', 'Lovely Spam', 'Wonderful Spam'])
    spamwriter.writerow(['FAR_WEST']+ ['Max Load','Year'])
    
#exapmle outfile
#Spam,Spam,Spam,Spam,Spam,Baked Beans
#Spam,Lovely Spam,Wonderful Spam
#FAR_WEST,Max Load,Year 


### read xls file


In [20]:
#pip install xlrd
import xlrd
datedir = "/Users/weidian1/Documents/Study/nanodegreee/P3"
datafile = "2013_ERCOT_Hourly_Load_Data.xls"


def parse_file(datedir,datafile):
    data = os.path.join(datedir, datafile)
    workbook = xlrd.open_workbook(data)
    sheet = workbook.sheet_by_index(0)

    data = [[sheet.cell_value(r, col) 
                for col in range(sheet.ncols)] 
                    for r in range(sheet.nrows)]

    print "\nList Comprehension"
    print "data[3][2]:",
    print data[3][2]

    print "\nCells in a nested loop:"    
    for row in range(sheet.nrows):
        for col in range(sheet.ncols):
            if row == 50:
                print sheet.cell_value(row, col),


    ### other useful methods:
    print "\nROWS, COLUMNS, and CELLS:"
    print "Number of rows in the sheet:", 
    print sheet.nrows
    print "Type of data in cell (row 3, col 2):", 
    print sheet.cell_type(3, 2)
    print "Value in cell (row 3, col 2):", 
    print sheet.cell_value(3, 2)
    print "Get a slice of values in column 3, from rows 1-3:"
    print sheet.col_values(3, start_rowx=1, end_rowx=4)

    print "\nDATES:"
    print "Type of data in cell (row 1, col 0):", 
    print sheet.cell_type(1, 0)
    exceltime = sheet.cell_value(1, 0)
    print "Time in Excel format:",
    print exceltime
    print "Convert time to a Python datetime tuple, from the Excel float:",
    print xlrd.xldate_as_tuple(exceltime, 0)

    return data

data = parse_file(datedir,datafile)


List Comprehension
data[3][2]: 1036.088697

Cells in a nested loop:
41277.0833333 9238.73731 1438.20528 1565.442856 916.708348 14010.903488 3027.98334 6165.211119 1157.741663 37520.933404 
ROWS, COLUMNS, and CELLS:
Number of rows in the sheet: 7296
Type of data in cell (row 3, col 2): 2
Value in cell (row 3, col 2): 1036.088697
Get a slice of values in column 3, from rows 1-3:
[1411.7505669999982, 1403.4722870000019, 1395.053150000001]

DATES:
Type of data in cell (row 1, col 0): 3
Time in Excel format: 41275.0416667
Convert time to a Python datetime tuple, from the Excel float: (2013, 1, 1, 1, 0, 0)


In [11]:
# -*- coding: utf-8 -*-
'''
Find the time and value of max load for each of the regions
COAST, EAST, FAR_WEST, NORTH, NORTH_C, SOUTHERN, SOUTH_C, WEST
and write the result out in a csv file, using pipe character | as the delimiter.

An example output can be seen in the "example.csv" file.
'''

import xlrd
import os
import csv
from zipfile import ZipFile
datadir = "/Users/weidian1/Documents/Study/nanodegreee/P3"
datafile = "2013_ERCOT_Hourly_Load_Data.xls"
outfile = "2013_Max_Loads.csv"


def open_zip(datadir,datafile):
    data = os.path.join(datadir, datafile)
    with ZipFile('{0}.zip'.format(data), 'r') as myzip:
        myzip.extractall()


def parse_file(datadir,datafile):
    datafile = os.path.join(datadir, datafile)
    workbook = xlrd.open_workbook(datafile)
    sheet = workbook.sheet_by_index(0)
    data = {}
    # process all rows that contain station data
    for n in range (1, 9):
        station = sheet.cell_value(0, n)
        cv = sheet.col_values(n, start_rowx=1, end_rowx=None)

        maxval = max(cv)
        maxpos = cv.index(maxval) + 1
        maxtime = sheet.cell_value(maxpos, 0)
        realtime = xlrd.xldate_as_tuple(maxtime, 0)
        data[station] = {"maxval": maxval,
                         "maxtime": realtime}
    return data

def save_file(data, filename):
    with open(filename, "w") as f:
        w = csv.writer(f, delimiter='|')
        w.writerow(["Station", "Year", "Month", "Day", "Hour", "Max Load"])
        for s in data:
            year, month, day, hour, _ , _= data[s]["maxtime"]
            w.writerow([s, year, month, day, hour, data[s]["maxval"]])
    
def test():
#    open_zip(datedir,datafile)
    data = parse_file(datadir,datafile)
    save_file(data, outfile)

    number_of_rows = 0
    stations = []

    ans = {'FAR_WEST': {'Max Load': '2281.2722140000024',
                        'Year': '2013',
                        'Month': '6',
                        'Day': '26',
                        'Hour': '17'}}
    correct_stations = ['COAST', 'EAST', 'FAR_WEST', 'NORTH',
                        'NORTH_C', 'SOUTHERN', 'SOUTH_C', 'WEST']
    fields = ['Year', 'Month', 'Day', 'Hour', 'Max Load']

    with open(outfile) as of:
        csvfile = csv.DictReader(of, delimiter="|")
        for line in csvfile:
            station = line['Station']
            if station == 'FAR_WEST':
                for field in fields:
                    # Check if 'Max Load' is within .1 of answer
                    if field == 'Max Load':
                        max_answer = round(float(ans[station][field]), 1)
                        max_line = round(float(line[field]), 1)
                        assert max_answer == max_line

                    # Otherwise check for equality
                    else:
                        assert ans[station][field] == line[field]

            number_of_rows += 1
            stations.append(station)

        # Output should be 8 lines not including header
        assert number_of_rows == 8

        # Check Station Names
        assert set(stations) == set(correct_stations)

        
if __name__ == "__main__":
    test()


### Get JSON fomart data

In [21]:
# To experiment with this code freely you will have to run this code locally.
# Take a look at the main() function for an example of how to use the code.
# We have provided example json output in the other code editor tabs for you to
# look at, but you will not be able to run any queries through our UI.
import json
import requests


BASE_URL = "http://musicbrainz.org/ws/2/"
ARTIST_URL = BASE_URL + "artist/"

# query parameters are given to the requests.get function as a dictionary; this
# variable contains some starter parameters.
query_type = {  "simple": {},
                "atr": {"inc": "aliases+tags+ratings"},
                "aliases": {"inc": "aliases"},
                "releases": {"inc": "releases"}}


def query_site(url, params, uid="", fmt="json"):
    # This is the main function for making queries to the musicbrainz API.
    # A json document should be returned by the query.
    params["fmt"] = fmt
    r = requests.get(url + uid, params=params)
    print "requesting", r.url

    #检测响应状态码
    if r.status_code == requests.codes.ok:  
        return r.json()
    else:
        r.raise_for_status()


def query_by_name(url, params, name):
    # This adds an artist name to the query parameters before making
    # an API call to the function above.
    params["query"] = "artist:" + name
    return query_site(url, params)


def pretty_print(data, indent=4):
    # After we get our output, we can format it to be more readable
    # by using this function.
    if type(data) == dict:
        print json.dumps(data, indent=indent, sort_keys=True)
    else:
        print data


def main():
    '''
    Modify the function calls and indexing below to answer the questions on
    the next quiz. HINT: Note how the output we get from the site is a
    multi-level JSON document, so try making print statements to step through
    the structure one level at a time or copy the output to a separate output
    file.
    '''
    results = query_by_name(ARTIST_URL, query_type["simple"], "Nirvana")
    pretty_print(results)

    artist_id = results["artists"][1]["id"]
    print "\nARTIST:"
    pretty_print(results["artists"][1])

    artist_data = query_site(ARTIST_URL, query_type["releases"], artist_id)
    releases = artist_data["releases"]
    print "\nONE RELEASE:"
    pretty_print(releases[0], indent=2)
    release_titles = [r["title"] for r in releases]

    print "\nALL TITLES:"
    for t in release_titles:
        print t


if __name__ == '__main__':
    main()


requesting http://musicbrainz.org/ws/2/artist/?query=artist%3ANirvana&fmt=json
{
    "artists": [
        {
            "aliases": [
                {
                    "begin-date": null, 
                    "end-date": null, 
                    "locale": null, 
                    "name": "Nirvana US", 
                    "primary": null, 
                    "sort-name": "Nirvana US", 
                    "type": null
                }
            ], 
            "area": {
                "id": "489ce91b-6658-3307-9877-795b68554c98", 
                "name": "United States", 
                "sort-name": "United States"
            }, 
            "begin-area": {
                "id": "a640b45c-c173-49b1-8030-973603e895b5", 
                "name": "Aberdeen", 
                "sort-name": "Aberdeen"
            }, 
            "country": "US", 
            "disambiguation": "90s US grunge band", 
            "id": "5b11f4ce-a62d-471e-81fc-a69a8278c7da", 
            "life-span

In [29]:
import json
import requests


BASE_URL = "http://musicbrainz.org/ws/2/"
ARTIST_URL = BASE_URL + "artist/"

# query parameters are given to the requests.get function as a dictionary; this
# variable contains some starter parameters.
query_type = {  "simple": {},
                "atr": {"inc": "aliases+tags+ratings"},
                "aliases": {"inc": "aliases"},
                "releases": {"inc": "releases"}}


def query_site(url, params, uid="", fmt="json"):
    # This is the main function for making queries to the musicbrainz API.
    # A json document should be returned by the query.
    params["fmt"] = fmt
    r = requests.get(url + uid, params=params)
    print "requesting", r.url

    #检测响应状态码
    if r.status_code == requests.codes.ok:  
        return r.json()
    else:
        r.raise_for_status()


def query_by_name(url, params, name):
    # This adds an artist name to the query parameters before making
    # an API call to the function above.
    params["query"] = "artist:" + name
    return query_site(url, params)


def pretty_print(data, indent=4):
    # After we get our output, we can format it to be more readable
    # by using this function.
    if type(data) == dict:
        print json.dumps(data, indent=indent, sort_keys=True)
    else:
        print data


def main():
    '''
    Modify the function calls and indexing below to answer the questions on
    the next quiz. HINT: Note how the output we get from the site is a
    multi-level JSON document, so try making print statements to step through
    the structure one level at a time or copy the output to a separate output
    file.
    '''
    results = query_by_name(ARTIST_URL, query_type["simple"], "BEATLES")
#    pretty_print(results)

    artist_id = results["artists"][1]["id"]
#    print "\nARTIST:"
#    pretty_print(results["artists"][1])

    artist_data = query_site(ARTIST_URL, query_type["aliases"], artist_id)
    releases = artist_data["aliases"]
#    print "\nONE RELEASE:"
    print releases
#    pretty_print(releases[0], indent=2)
#    release_titles = [r["title"] for r in releases]

#    print "\nALL TITLES:"
#    for t in release_titles:
#        print t

if __name__ == '__main__':
    main()

requesting http://musicbrainz.org/ws/2/artist/?query=artist%3ABEATLES&fmt=json
requesting http://musicbrainz.org/ws/2/artist/bc569a61-dd62-4758-86c6-e99dcb1fdda6?fmt=json&inc=aliases
[]


In [42]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-
"""
This exercise shows some important concepts that you should be aware about:
- using codecs module to write unicode files
- using authentication with web APIs
- using offset when accessing web APIs

To run this code locally you have to register at the NYTimes developer site 
and get your own API key. You will be able to complete this exercise in our UI
without doing so, as we have provided a sample result.

Your task is to process the saved file that represents the most popular
articles (by view count) from the last day, and return the following data:
- list of dictionaries, where the dictionary key is "section" and value is "title"
- list of URLs for all media entries with "format": "Standard Thumbnail"

All your changes should be in the article_overview function.
The rest of functions are provided for your convenience, if you want to access
the API by yourself.
"""
import json
import codecs
import requests

URL_MAIN = "http://api.nytimes.com/svc/"
URL_POPULAR = URL_MAIN + "mostpopular/v2/"
API_KEY = { "popular": "d47c4bf33872431f80cb97e2e8a540d3",
            "article": "article"}


def get_from_file(kind, period):
    filename = "popular-{0}-{1}.json".format(kind, period)
    with open(filename, "r") as f:
        return json.loads(f.read())


def article_overview(kind, period):
    data = get_from_file(kind, period)
    titles = []
    urls =[]
    for asset in data:
        titles.append({asset["section"]: asset["title"]})
        for med in asset['media']:
            for e in med['media-metadata']:
                if e['format'] == "Standard Thumbnail":
                     urls.append(e['url'])
    return (titles, urls)

def query_site(url, target, offset):
    # This will set up the query with the API key and offset
    # Web services often use offset paramter to return data in small chunks
    # NYTimes returns 20 articles per request, if you want the next 20
    # You have to provide the offset parameter
    if API_KEY["popular"] == "" or API_KEY["article"] == "":
        print "You need to register for NYTimes Developer account to run this program."
        print "See Intructor notes for information"
        return False
    params = {"api-key": API_KEY[target], "offset": offset}
    r = requests.get(url, params = params)

    if r.status_code == requests.codes.ok:
        return r.json()
    else:
        r.raise_for_status()


def get_popular(url, kind, days, section="all-sections", offset=0):
    # This function will construct the query according to the requirements of the site
    # and return the data, or print an error message if called incorrectly
    if days not in [1,7,30]:
        print "Time period can be 1,7, 30 days only"
        return False
    if kind not in ["viewed", "shared", "emailed"]:
        print "kind can be only one of viewed/shared/emailed"
        return False

    url += "most{0}/{1}/{2}.json".format(kind, section, days)
    data = query_site(url, "popular", offset)

    return data


def save_file(kind, period):
    # This will process all results, by calling the API repeatedly with supplied offset value,
    # combine the data and then write all results in a file.
    data = get_popular(URL_POPULAR, "viewed", 1)
    num_results = data["num_results"]
    full_data = []
    with codecs.open("popular-{0}-{1}.json".format(kind, period), encoding='utf-8', mode='w') as v:
        for offset in range(0, num_results, 20):        
            data = get_popular(URL_POPULAR, kind, period, offset=offset)
            full_data += data["results"]
        
        v.write(json.dumps(full_data, indent=2))


def test():
    titles, urls = article_overview("viewed", 1)
    assert len(titles) == 20
    assert len(urls) == 30
    assert titles[2] == {'Opinion': 'Professors, We Need You!'}
    assert urls[20] == 'http://graphics8.nytimes.com/images/2014/02/17/sports/ICEDANCE/ICEDANCE-thumbStandard.jpg'


if __name__ == "__main__":
    save_file(, 1)

ConnectionError: HTTPConnectionPool(host='api.nytimes.com', port=80): Max retries exceeded with url: /svc/mostpopular/v2/mostviewed/all-sections/1.json?api-key=d47c4bf33872431f80cb97e2e8a540d3&offset=0 (Caused by NewConnectionError('<requests.packages.urllib3.connection.HTTPConnection object at 0x105217f50>: Failed to establish a new connection: [Errno 60] Operation timed out',))

In [ ]:
#Here's your API Key for the Article Search API: d47c4bf33872431f80cb97e2e8a540d3

In [20]:
"""当python要做编码转换的时候，会借助于内部的编码，转换过程是这样的：
原有编码 -> 内部编码 -> 目的编码
python的内部是使用unicode来处理的，但是unicode的使用需要考虑的是它的编码格式有两种，一是UCS-2，它一共有65536个码位，另一种是UCS-4，它有2147483648g个码位。"""
import sys
print sys.maxunicode
#如果输出的值为65535,那么就是UCS-2,如果输出是1114111就是UCS-4编码。

65535


In [40]:
import codecs, sys

print '-'*60
# 创建gb2312编码器
look  = codecs.lookup("gbk")
# 创建utf-8编码器
look2 = codecs.lookup("utf-8")

a = "我爱北京"

print len(a), a
# 把a编码为内部的unicode, 但为什么方法名为decode呢，我的理解是把gbk的字符串解码为unicode
b = look2.decode(a)
# 返回的b[0]是数据，b[1]是长度，这个时候的类型是unicode了
print b[1], b[0], type(b[0])
# 把内部编码的unicode转换为gb2312编码的字符串，encode方法会返回一个字符串类型
b2 = look2.encode(b[0])
# 发现不一样的地方了吧？转换回来之后，字符串长度由12变为了4! 现在的返回的长度才是真正的字数，原来的是字节数
print b2[1], b2[0], type(b2[0])
# 虽然上面返回了字数，但并不意味着用len求b2[0]的长度就是4了，仍然还是12，仅仅是codecs.encode会统计字数
print len(b2[0])

------------------------------------------------------------
12 我爱北京
12 我爱北京 <type 'unicode'>
4 我爱北京 <type 'str'>
12


# Xml


API  
[XPath_sytax](http://www.w3schools.com/xsl/xpath_syntax.asp)  
[xml.etree.elementtree](https://docs.python.org/2/library/xml.etree.elementtree.html)
[BeautifulSoup](https://www.crummy.com/software/BeautifulSoup/bs4/doc/)

In [56]:
import xml.etree.ElementTree as ET
import pprint
import codecs

tree = ET.parse(codecs.open("exampleResearchArticle.xml", encoding="UTF-8"))
root = tree.getroot()

title = root.find('./fm/bibl/title')
title_text = ""
for p in title:
    title_text += p.text
print "\nTitle:\n",title_text

print "\nAuthor email addresses:"
for a in root.findall('./fm/bibl/aug/au'):
    email = a.find('email')
    if email is not None:
        print email.text


Title:
Standardization of the functional syndesmosis widening by dynamic U.S examination

            

Author email addresses:
omer@extremegate.com
mcarmont@hotmail.com
laver17@gmail.com
nyska@internet-zahav.net
kammarh@gmail.com
gideon.mann.md@gmail.com
barns.nz@gmail.com
eukots@gmail.com


In [35]:
from bs4 import BeautifulSoup
import requests

# 搜索框中的选项  [{名：缩写},...]
def options(soup, id):
    option_values = []
    carrier_list = soup.find(id = id)
    #soup.find("a", id="link3")     tag='a',attribution id='link3'
    for option in carrier_list.find_all('option'): #tag 是 option的所有element
        option_values.append(option['value']) #响应的 value属性的值
    return option_values

#格式化输出 print("I'm %s. I'm %d year old" % ('Vamei', 99))
def print_list(label, codes):
    print "\n%s:"%label
    for c in codes:
        print c

#提交表单， 
def post():        
    #保留会话，get和post用一个会话
    s = requests.Session()

    r = s.get("http://www.transtats.bts.gov/Data_Elements.aspx?Data=2")  
    soup = BeautifulSoup(r.text)
    viewstate = soup.find('input',id='__VIEWSTATE')['value']
    eventvalidation = soup.find('input',id='__EVENTVALIDATION')['value']

    r = s.post("http://www.transtats.bts.gov/Data_Elements.aspx?Data=2",
                  data = {"__EVENTTARGET":"",
                          "__EVENTARGUMENT":"",
                          "__VIEWSTATE":viewstate,
                          "__EVENTVALIDATION":eventvalidation,
                          "CarrierList":CarrierList['Virgin America'],
                          "AirportList":AirportList[' - Boston, MA: Logan International'],
                          "Submit":'Submit'})
def write_html(file_name):
    with open(file_name,"w") as f:
        f.write(r.text)



def main():
    # BeautifulSoup(markup, "html.parser")  parser library：‘lxml’，‘html.parser’ ...
    soup = BeautifulSoup(open("virgin_and_logan_airport.html"))
    CarrierList = options(soup,'CarrierList')
    #print_list("Carries",codes)
    CarrierList = options(soup,'AirportList')
    #print_list("Airpirts",codes)
    post()
    write_html("virgin_and_logan_airport_out.html")
    
    

#if __name__ == '__main__':
#    main()

In [ ]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-
# 
import xml.etree.cElementTree as ET
#dict subclass that calls a factory function to supply missing values。
#defaultdict(default_factory), 按default_factory给key生成默认的value
from collections import defaultdict
import re

osm_file = open("chicago.osm", "r")

#设置匹配模式：非空白字符，以1个或0个‘.’结束， 忽略大小写
street_type_re = re.compile(r'\S+\.?$', re.IGNORECASE)
street_types = defaultdict(int) #默认value=0


def audit_street_type(street_types, street_name):
    m = street_type_re.search(street_name)
    if m:
        street_type = m.group()

        street_types[street_type] += 1

def print_sorted_dict(d):
    keys = d.keys()
    keys = sorted(keys, key=lambda s: s.lower())
    for k in keys:
        v = d[k]
        print "%s: %d" % (k, v)

def is_street_name(elem):
    return (elem.tag == "tag") and (elem.attrib['k'] == "addr:street")

def audit():
    #ET.iterparse 不一次全部加载进内存，
    for event, elem in ET.iterparse(osm_file):
        if is_street_name(elem):
            audit_street_type(street_types, elem.attrib['v'])
    print_sorted_dict(street_types)

if __name__ == '__main__':
    audit()

In [2]:
from collections import defaultdict
s = 'mississippi'
d = defaultdict(int)
for k in s:
     d[k] += 1

d.items()
#[('i', 4), ('p', 2), ('s', 4), ('m', 1)]


[('i', 4), ('p', 2), ('s', 4), ('m', 1)]

In [3]:
dict(d)

{'i': 4, 'm': 1, 'p': 2, 's': 4}

### Blueprint

In [ ]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-
#
import xml.etree.cElementTree as ET
#dict subclass that calls a factory function to supply missing values。
#defaultdict(default_factory), 按default_factory给key生成默认的value
from collections import defaultdict
import re

osm_file = open("chicago.osm", "r")

#设置匹配模式：非空白字符，以1个或0个‘.’结束， 忽略大小写
street_type_re = re.compile(r'\S+\.?$', re.IGNORECASE)
street_types = defaultdict(int) #默认value=0


def audit_street_type(street_types, street_name):
    m = street_type_re.search(street_name)
    if m:
        street_type = m.group()

        street_types[street_type] += 1

def print_sorted_dict(d):
    keys = d.keys()
    keys = sorted(keys, key=lambda s: s.lower())
    for k in keys:
        v = d[k]
        print "%s: %d" % (k, v)

def is_street_name(elem):
    return (elem.tag == "tag") and (elem.attrib['k'] == "addr:street")

def audit():
    #ET.iterparse 不一次全部加载进内存，
    for event, elem in ET.iterparse(osm_file):
        if is_street_name(elem):
            audit_street_type(street_types, elem.attrib['v'])
    print_sorted_dict(street_types)

if __name__ == '__main__':
    audit()


## MongoDB
PyMongo translate a python dictinoary into BSON  
[PyMongo_doc](https://docs.mongodb.com/getting-started/python/query/)  
[PyMongo](http://api.mongodb.com/python/2.8/tutorial.html#bulk-inserts)  
[mongodb_manual](https://docs.mongodb.com/manual/reference/command/insert/#dbcmd.insert)  
[Query_Project](https://docs.mongodb.com/manual/reference/operator/query/)

Methods:get data into MongoDB  
1. cleaning data in python, then put into MongoDB  
2. Json data,then bulk import MongoDB (using mongoimport)  
mongoimport -db dbname -c collectionname --file input-file.json  
[mongoimport](https://docs.mongodb.com/manual/reference/program/mongoimport/)  
ps:If no hostname and credentials are supplied, mongoimport will try to connect to the default localhost:27017

In [ ]:
"""插入数据docment  [PyMongo translate a python dictinoary into BSON encoding and send it across to the database]
Insert a document into a collection named autos.
The operation will create the collection if the collection does not currently exist.
"""
from pymongo import MongoClient
    client = MongoClient('localhost:27017')
    db = client['example']
tesla_s = {...}
db.autos.insert(tesla_s) #autos 是一个 collection
num_autos = db.autos.find().count()

In [ ]:
#!/usr/bin/env python
"""字段筛选
"""

# Do not edit code below this line in the online code editor.
# Code here is for local use on your own computer.
def get_db(db_name):
    # For local use
    from pymongo import MongoClient
    client = MongoClient('localhost:27017')
    db = client[db_name]
    return db

def find_porsche(db, query):
    # For local use
    return db.autos.find({"manufacturer":"Porsche","class":"mid-size car"})


if __name__ == "__main__":
    # For local use
    db = get_db('examples')
    query = porsche_query()
    results = find_porsche(db, query)

    print "Printing first 3 results\n"
    import pprint
    for car in results[:3]:
        pprint.pprint(car)

In [ ]:
#projection 是在query查询结果之后的筛选，query类似where限制条件，projection类似select 的选择字段
def find():
    query ={"manufacturer":"Porsche","class":"mid-size car"}
    projection ={"_id":0,"name":1} #不返回id，返回name
    result = db.autos.find(query,projection)
    return result

operator
1. range operator
2. if field exists 
3. regex operator    
    Live RegEx tester at [regexpal.com](http://www.regexpal.com/)    
    MongoDB [$regex Manual](https://docs.mongodb.com/manual/reference/operator/query/regex/)  
    Official Python [Regular Expression HOWTO](https://docs.python.org/2/howto/regex.html)  
    [Another good Python Regular Expressions page](https://developers.google.com/edu/python/regular-expressions?csw=1)  
4. in / all operator
5. query array结构化数据 using scalars标量(like,int,strings)
6. Dot notation

Updates Methods: 
1. db.cities.save(city)  
2. city = db.cities.update({"name":"Munchen","country":"Germany"},{"$set":{"isoCountryCode":"DEU"}}) 

3. city = db.cities.update({"name":"Munchen","country":"Germany"},{"$unset":{"isoCountryCode":""}})  
4. city = db.cities.update({"country":"Germany"},{"$set":{"isoCountryCode":"DEU"}},multi = True) 

In [ ]:
#save
city = db.cities.find_one({})
city["isoCountryCode"] = "DEU"
db.cities.save(city)  #可以指定 _id(新增/更新)/系统分配

#update  $set
city = db.cities.update({"name":"Munchen","country":"Germany"},{"$set":{"isoCountryCode":"DEU"}}) #无isoCountryCode：新增/有：更新

#update $unset 有：删除/无：不修改

#multi update 
city = db.cities.update({"country":"Germany"},{"$set":{"isoCountryCode":"DEU"}},multi = True) 

In [23]:
#DON‘T DO THIS  符合查询条件的docment 全部替换成 {"isoCountryCode":"DEU"}
city = db.cities.update({"name":"Munchen","country":"Germany"},{"isoCountryCode":"DEU"}) 

NameError: name 'db' is not defined

Removing document

In [ ]:
db.cities.remove() 
db.cities.remove({"name":"Chicago"}) #删除满足条件的documents
db.cities.drop() #删除整个collection和相关的元数据，比如索引

### aggregation pipeline ：operate one by one
[mongodb_aggregation_operator](https://docs.mongodb.com/manual/reference/operator/aggregation-pipeline/)  

operation|fuction
--|--|
\$group| divide
\$sort  |sort
\$project  |shaping: pass specified fields(existing or newly computed) to the next stage.  
\$match |filtering
\$skip  |skip top
\$limit  |get top
\$unwind 可以将一个document按array中不同的value分解成多个documents  

[group operators](https://docs.mongodb.com/manual/reference/operator/aggregation/group/)  

\$sum  
\$first  
\$last  
\$max  
\$min  
\$avg  
\$push  
\$addToSet  

Conditional Aggregation Operators
ifNull:null、missing，返回替代值；否则，返回原value


     {
         "$project": {
            "item": 1,
            "description": { "$ifNull": [ "$description", "Unspecified" ] }
         }
      }



In [ ]:
#$group;$month等 $表示调用函数
# _id;totalPrice;averageQuantity;count等 输出的field
#_id 分组的依据，不可省略，可为NULL或固值（在对整体求平均时）
#$group  不带order功能
db.sales.aggregate(
   [
      {
        "$group" : {
           "_id" : { "month": { "$month": "$date" }, "day": { "$dayOfMonth": "$date" }, "year": { "$year": "$date" } },
           "totalPrice": { "$sum": { "$multiply": [ "$price", "$quantity" ] } },
           "averageQuantity": { "$avg": "$quantity" },
           "count": { "$sum": 1 }
        }
      }
      {
        "$sort" :{ "averageQuantity": -1}
        }
   ]
)

In [ ]:
# $ 用于1.表示调用函数 2.取该filed name的value，而非字符串
# "screen_name":"$user.screen_name" 第一个screen_name 表示结果的field name（不存在，则新建）

pipeline = [
                {"$match":{"user.time_zone":"Brasilia",
                           "user.statuses_count":{"$gte":100},
                            "user.followers_count":{"$gt":0}}},
                {"$project":{"screen_name":"$user.screen_name",
                             "tweets":"$user.statuses_count",
                             "followers":"$user.followers_count"}},
                {"$sort":{"followers":-1}},
                {"$limit":1}
                ]
#match  multi conditions
{"$match":{"country":"India","lon":{"$gte":75,"$let":80}}}

In [ ]:
    pipeline = [{"$match":{"country":"India"}},
                {"$unwind":"$isPartOf"},
                {"$group":{"_id":"$isPartOf","count":{"$sum":1}}},
                {"$sort":{"count":-1}},
                {"$limit":1} 

In [ ]:
# $addToSet
{
       $group:
         {
           _id: { day: { $dayOfYear: "$date"}, year: { $year: "$date" } },
           itemsSold: { $addToSet: "$item" }
         }
     }

#output
{ "_id" : { "day" : 46, "year" : 2014 }, "itemsSold" : [ "xyz", "abc" ] }
{ "_id" : { "day" : 34, "year" : 2014 }, "itemsSold" : [ "xyz", "jkl" ] }
{ "_id" : { "day" : 1, "year" : 2014 }, "itemsSold" : [ "abc" ] }

In [ ]:
# $push
{
       $group:
         {
           _id: { day: { $dayOfYear: "$date"}, year: { $year: "$date" } },
           itemsSold: { $push:  { item: "$item", quantity: "$quantity" } }
         }
     }

#output
{
   "_id" : { "day" : 46, "year" : 2014 },
   "itemsSold" : [
      { "item" : "abc", "quantity" : 10 },
      { "item" : "xyz", "quantity" : 10 },
      { "item" : "xyz", "quantity" : 5 },
      { "item" : "xyz", "quantity" : 10 }
   ]
}
{
   "_id" : { "day" : 34, "year" : 2014 },
   "itemsSold" : [
      { "item" : "jkl", "quantity" : 1 },
      { "item" : "xyz", "quantity" : 5 }
   ]
}

## index


### 多个运算阶段

## 附录
### josn data <--> python data

In [14]:
"""json 模块提供了一种很简单的方式来编码和解码JSON数据。 
其中两个主要的函数是 json.dumps() 和 json.loads(),要比其他序列化函数库如pickle的接口少得多。 """
#下面演示如何将一个Python数据结构转换为JSON：
import json

data = {
    'name' : 'ACME',
    'shares' : 100,
    'price' : 542.23
}

json_str = json.dumps(data)
#下面演示如何将一个JSON编码的字符串转换回一个Python数据结构：
data = json.loads(json_str)


#如果你要处理的是文件而不是字符串，你可以使用 json.dump() 和 json.load() 来编码和解码JSON数据。例如：
# Writing JSON data
with open('data.json', 'w') as f:
    json.dump(data, f)

# Reading data back
with open('data.json', 'r') as f:
    data = json.load(f)

### Re 正则表达式

[Python正则表达式指南](http://www.cnblogs.com/huxi/archive/2010/07/04/1771073.html)  
[官方re](https://docs.python.org/2/library/re.html)

In [ ]:
#设置一种匹配模式，再去匹配
The sequence
prog = re.compile(pattern)
result = prog.match(string)

#直接用一种模式匹配，一次性
is equivalent to
result = re.match(pattern, string)

#不含字母数字
if (re.match('[^a-z0-9]', row[k], re.IGNORECASE)) 

In [10]:
from collections import defaultdict
s = 'mississippi'
d = defaultdict(int)
for k in s:
     d[k] += 1
d.items()
#[('i', 4), ('p', 2), ('s', 4), ('m', 1)]

[('i', 4), ('p', 2), ('s', 4), ('m', 1)]

In [14]:
import re
m = re.match(r"(\w+) (\w+)", "Isaac Newton, physicist")
print m.group(0)       # The entire match
#'Isaac Newton'
print m.group(1)       # The first parenthesized subgroup.
#'Isaac'
print m.group(2)       # The second parenthesized subgroup.
#'Newton'
print m.group(1, 2)    # Multiple arguments give us a tuple.
#('Isaac', 'Newton')

Isaac Newton
Isaac
Newton
('Isaac', 'Newton')


In [15]:
m.group()

'Isaac Newton'

### get sample_file

In [ ]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-

import xml.etree.ElementTree as ET  # Use cElementTree or lxml if too slow

OSM_FILE = "some_osm.osm"  # Replace this with your osm file
SAMPLE_FILE = "sample.osm"

k = 10 # Parameter: take every k-th top level element

def get_element(osm_file, tags=('node', 'way', 'relation')):
    """Yield element if it is the right type of tag

    Reference:
    http://stackoverflow.com/questions/3095434/inserting-newlines-in-xml-file-generated-via-xml-etree-elementtree-in-python
    """
    context = iter(ET.iterparse(osm_file, events=('start', 'end')))
    _, root = next(context)
    for event, elem in context:
        if event == 'end' and elem.tag in tags:
            yield elem
            root.clear()


with open(SAMPLE_FILE, 'wb') as output:
    output.write('<?xml version="1.0" encoding="UTF-8"?>\n')
    output.write('<osm>\n  ')

    # Write every kth top level element
    for i, element in enumerate(get_element(OSM_FILE)):
        if i % k == 0:
            output.write(ET.tostring(element, encoding='utf-8'))

    output.write('</osm>')

### SET

In [28]:
a = set(['boy'])
a.add('gril')
a.add('w')
a

{'boy', 'gril', 'w'}

In [29]:
a = set('boy')
a.add('gril')
a.add('w')
a

{'b', 'gril', 'o', 'w', 'y'}

In [6]:
a = set()
a.add("girl")
a.add('boy')
a

{'boy', 'girl'}

In [40]:
area = '{3.43173e+07|3.432e+07}'

In [45]:
max([float(a) for a in (area[1:-1].split("|"))])

34320000.0

In [46]:
'a'.split(" ")

['a']

In [22]:
import re
s = ' acewuo ave.'
street_type = re.compile(r'\b\S+\.?$',re.IGNORECASE) 
m = street_type.search(s)
m.group()

'ave.'